# Deployment Dashboard

In [4]:
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn
import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier

In [3]:
#!pip install gunicorn

     ---------------------------------------- 79.5/79.5 kB 1.5 MB/s eta 0:00:00


In [5]:
#!pip install pipreqs

In [2]:
#!pip install fastapi uvicorn

import joblib
import uvicorn
import tkinter

In [3]:
#!pip install mlflow --user
#!pip install fastapi
#!pip install streamlit_shap

In [4]:
import mlflow
import pickle

In [5]:
#!pip install sklego

In [6]:
from sklego.meta import Thresholder

In [8]:
model_test=pd.read_csv("C:\\Users\\USER\\Documents\\P7dataset\\features_test.csv")
model_test.head()

,SK_ID_CURR,TARGET,DAYS_EMPLOYED,EXT_SOURCE_3,DAYS_BIRTH,APARTMENTS_AVG,DAYS_REGISTRATION,CREDIT_TYPE_Microloan,PREV_DAYS_DECISION_MIN
0,100001,NaN,-2329,0.159520,-19241,0.0660,-5170.0,0.0,-1740.0
1,100005,NaN,-4469,0.432962,-18064,0.0000,-9118.0,0.0,-536.0
2,100013,NaN,-4458,0.610991,-20038,0.0000,-2175.0,0.0,-837.5
3,100028,NaN,-1866,0.612704,-13976,0.3052,-2000.0,0.0,-1124.2
4,100038,NaN,-2191,0.000000,-13040,0.0000,-4000.0,0.0,-466.0


In [12]:
model_test = model_test.drop(columns='TARGET')

In [9]:
model_train=pd.read_csv("C:\\Users\\USER\\Documents\\P7dataset\\features_train.csv")
model_train.head()

,SK_ID_CURR,TARGET,DAYS_EMPLOYED,EXT_SOURCE_3,DAYS_BIRTH,APARTMENTS_AVG,DAYS_REGISTRATION,CREDIT_TYPE_Microloan,PREV_DAYS_DECISION_MIN
0,333721,0,-688,0.651260,-11763,0.2938,-500.0,0.0,-496.666667
1,110157,0,365243,0.638044,-23199,0.0763,-4264.0,0.0,-860.181818
2,276815,0,-3432,0.108226,-19165,0.2361,-4524.0,0.0,-1147.333333
3,353213,0,-1154,0.194068,-10416,0.0000,-4766.0,0.0,-1501.100000
4,447111,0,-3256,0.441836,-12125,0.0000,-1913.0,0.0,0.000000


In [13]:
model_test.columns=["IdClient","DaysEmployed", "ExtSource3",
                                   "DaysBirth", "ApartmentAvg", "DaysRegistration", "CreditTypeMicroloan", "PrevDaysDecisionMin"]

model_train.columns=["IdClient","Target", "DaysEmployed", "ExtSource3",
                                   "DaysBirth", "ApartmentAvg", "DaysRegistration", "CreditTypeMicroloan", "PrevDaysDecisionMin"]

In [28]:
model_train.to_csv("C:\\Users\\USER\\Documents\\P7dataset\\features_train.csv", index=False)
model_test.to_csv("C:\\Users\\USER\\Documents\\P7dataset\\features_test.csv", index=False)

In [16]:
d_types = dict(model_test.dtypes)
d_types

{'IdClient': dtype('int64'),
 'DaysEmployed': dtype('int64'),
 'ExtSource3': dtype('float64'),
 'DaysBirth': dtype('int64'),
 'ApartmentAvg': dtype('float64'),
 'DaysRegistration': dtype('float64'),
 'CreditTypeMicroloan': dtype('float64'),
 'PrevDaysDecisionMin': dtype('float64')}

In [17]:
type_dict = {}
for key, val in d_types.items():
    #print(key, val)\n",
    type_dict = {**type_dict, key: str(val)}
type_dict

{'IdClient': 'int64',
 'DaysEmployed': 'int64',
 'ExtSource3': 'float64',
 'DaysBirth': 'int64',
 'ApartmentAvg': 'float64',
 'DaysRegistration': 'float64',
 'CreditTypeMicroloan': 'float64',
 'PrevDaysDecisionMin': 'float64'}

In [8]:
%%writefile appfile.py

import pandas as pd
import numpy as np
import pickle
from fastapi import FastAPI
import uvicorn
import gunicorn
from typing import List
from pydantic import BaseModel
from sklego.meta import Thresholder

# define class which describes test data measurements
class class_testdata(BaseModel):
    
    DaysEmployed: float
    ExtSource3: float
    DaysBirth: float
    ApartmentAvg: float
    DaysRegistration: float
    CreditTypeMicroloan: float
    PrevDaysDecisionMin: float
    IdClient: int
    
    #class class_item(BaseModel):
    #    prediction: float        
    # app object creation
app = FastAPI()
pickle_in = open('C:\\Users\\USER\\Documents\\P7dataset\\mlflow_model\\model.pkl', 'rb') # load threshold model for probability prediction\n",
classifier = pickle.load(pickle_in)
# index route, opens automatically on
@app.get('/')
def index():
    return {'message': 'Welcome!'}

# route with a single parameter
@app.get('/{number}')
def get_name(name: int):
    return {'Welcome to page': {number}}

# make prediction functionality\n",
#@app.post('/predict', response_model=class_item) # the API's name
@app.post('/predict')
async def client_predict(data: class_testdata):
    data = data.dict()
    IdClient= data['IdClient']
    DaysEmployed= data['DaysEmployed']
    ExtSource3= data['ExtSource3']
    DaysBirth= data['DaysBirth']
    ApartmentAvg= data['ApartmentAvg']
    DaysRegistration= data['DaysRegistration']
    CreditTypeMicroloan= data['CreditTypeMicroloan']
    PrevDaysDecisionMin = data['PrevDaysDecisionMin']
    
    prediction = classifier.predict_proba([[DaysEmployed, ExtSource3,  
                                   DaysBirth, ApartmentAvg, DaysRegistration, CreditTypeMicroloan, PrevDaysDecisionMin]])
    
    prediction_default = prediction[0][0]
   
    return prediction_default
if __name__ == '__main__':
    uvicorn.run(app, host='127.0.0.1', port= 8000)

Overwriting appfile.py


 ##### terminal
    cd C:\\Users\\USER\\Documents\\P7dataset\\
    mlflow models serve -m mlflow_model/
    uvicorn appfile:app --reload
    streamlit run dashboard.py

In [2]:
%%writefile dashboard.py

import pandas as pd
import numpy as np
import streamlit as st
import requests
import json
from appfile import client_predict
import plotly

import plotly.graph_objects as go
#import plotly.express as px
from matplotlib import pyplot as plt
import tkinter

import pickle
import shap
from streamlit_shap import st_shap

# config
st.set_page_config(layout = 'wide')

reduce_header_height_style = """
    <style>
        div.block-container {padding-top:0rem;}
    </style>
"""
st.markdown(reduce_header_height_style, unsafe_allow_html=True)

prediction_data = pd.read_csv("C:\\Users\\USER\\Documents\\P7dataset\\features_test.csv")
#prediction_data = prediction_data.drop(columns='AmtIncomeTotal')
# loading the client info file prepared in modeling code
client_info = pd.read_csv('C:\\Users\\USER\\Documents\\P7dataset\\test_client_info.csv')

prediction_data_info = pd.merge(prediction_data, client_info, how='inner', left_on='IdClient', right_on='SK_ID_CURR')
#prediction_data_info = prediction_data_info.drop(columns='TARGET')
prediction_data_info.set_index('IdClient', inplace=True)

#avoid int64 erreur: convert to int
prediction_data_info['DaysEmployed'] = prediction_data_info['DaysEmployed'].astype('float')
prediction_data_info['DaysBirth'] = prediction_data_info['DaysBirth'].astype('float')

    #api_choice = st.sidebar.selectbox(
    #    'API name',
    #    ['MLflow'])


IdClient = st.sidebar.selectbox('Client ID to predict', [el for el in list(prediction_data_info.index)])

DaysEmployed= prediction_data_info.loc[IdClient, 'DaysEmployed']
ExtSource3= prediction_data_info.loc[IdClient, 'ExtSource3']
DaysBirth= prediction_data_info.loc[IdClient, 'DaysBirth']
ApartmentAvg= prediction_data_info.loc[IdClient, 'ApartmentAvg']
DaysRegistration= prediction_data_info.loc[IdClient, 'DaysRegistration']
CreditTypeMicroloan= prediction_data_info.loc[IdClient, 'CreditTypeMicroloan']
PrevDaysDecisionMin = prediction_data_info.loc[IdClient, 'PrevDaysDecisionMin']



Gender = prediction_data_info.loc[IdClient, 'CODE_GENDER']
Age = prediction_data_info.loc[IdClient, 'AGE']
Revenue = prediction_data_info.loc[IdClient, 'AMT_INCOME_TOTAL']
RevenueType = prediction_data_info.loc[IdClient, 'NAME_INCOME_TYPE']

Annuity = prediction_data_info.loc[IdClient, 'AMT_ANNUITY']
ContractType = prediction_data_info.loc[IdClient, 'NAME_CONTRACT_TYPE']
PreviousApp = prediction_data_info.loc[IdClient, 'PREVIOUS_APPLICATIONS']


inputs = {
        'DaysEmployed': DaysEmployed,
        'ExtSource3': ExtSource3,
        'DaysBirth': DaysBirth,
        'ApartmentAvg': ApartmentAvg,
        'DaysRegistration': DaysRegistration,
        'CreditTypeMicroloan': CreditTypeMicroloan,
        'PrevDaysDecisionMin': PrevDaysDecisionMin,
        'IdClient': IdClient
        }


@st.cache
def get_shap_data():

    # loading the trained model for compute shap value
    pickle_in = open('C:\\Users\\USER\\Documents\\P7dataset\\mlflow_model_0\\model.pkl', 'rb') # load no-threshold model for shap computation
    model = pickle.load(pickle_in)

    train_sample = pd.read_csv('C:\\Users\\USER\\Documents\\P7dataset\\features_train.csv')
    X = train_sample.drop(columns=['IdClient', 'Target'])
    y = train_sample['Target']
    model.fit(X,y)

    inputs_dict = {k:[v] for k,v in inputs.items()} # dictionary for a dataframe
    pd_inputs = pd.DataFrame(inputs_dict).drop(columns='IdClient') # ensure only input features, Id client/Target excluded

    explainer = shap.TreeExplainer(model['Classifier'])
    sv_explainer = explainer(pd_inputs) # apply treeExplainer on test instance
    return explainer, sv_explainer, pd_inputs


def main():
    fast_uri = "http://127.0.0.1:8000/predict"

    # sidebar
    st.sidebar.write('Client: ', IdClient)
    #if Gender == 'F':
    #    Gender == 'Female'
    #    st.sidebar.write('Gender:', 'Female')
    #else:
    #    Gender == 'Male'
    #    st.sidebar.write('Gender:', 'Male')
    st.sidebar.write('Gender:', Gender)
    st.sidebar.write('Age:', Age)
    st.sidebar.write('Revenu:', Revenue)
    st.sidebar.write('Revenu Type:', RevenueType)

    # main page
    #PageTitle='<p style="padding-top:0; font-family:courier; font-size: 42px; font-weight: bold; text-align: center;">Client Credit Default Prediction Dashbord</p>'
    #st.markdown(PageTitle, unsafe_allow_html=True)

   # #column1, column2, column3
    with open('style.css') as w:
        st.markdown(f'<style>{w.read()}</style>', unsafe_allow_html=True)

    col1, col2, col3 = st.columns(3)
    #value1 ='<p style="font-weight: bold; font-family:courier; font-size: 24px; text-align: center;">f'{PreviousApp}'</p>'
    #col1.subheader(f'{PreviousApp}')
    col1.markdown(f"""
                  <p style="font-weight: bold; font-family:courier; font-size: 24px; text-align: center;">{PreviousApp}</p>
                    """, unsafe_allow_html=True)
    col1.markdown('Previous Applications', unsafe_allow_html=True)
    #col2.subheader(f'{Annuity}')
    col2.markdown(f"""
                  <p style="font-weight: bold; font-family:courier; font-size: 24px; text-align: center;">{Annuity}</p>
                    """, unsafe_allow_html=True)
    col2.markdown('Annuity Amount', unsafe_allow_html=True)
    #col3.subheader(f'{ContractType}')
    col3.markdown(f"""
                  <p style="font-weight: bold; font-family:courier; font-size: 24px; text-align: center;">{ContractType}</p>
                    """, unsafe_allow_html=True)
    col3.markdown('Contract Type', unsafe_allow_html=True)


    #if st.sidebar.selectbox('Client ID to predict', [el for el in list(prediction_data.index)]):
    if st.sidebar.button('Predict'):
        # convert input dictionnary to json string
        inputs_json = json.dumps(inputs)

        # send api request
        headers = {"Content-Type": "application/json"}

        response = requests.post(url=fast_uri, data=inputs_json, headers= headers)

        if response.status_code != 200:
            raise Exception(
                "Request failed with status {}, {}".format(response.status_code, response.text))

        # convert string to float
        proba = np.array(response.text, dtype=float).item() # got np.ndarray, use item to get the first item which should return a float value
        #proba_flatten = proba.flatten()
        proba = round(proba, 2)
        st.sidebar.write(f'The default probability predicted is {proba}')

        # graphics plotting
        col1, col2 = st.columns([1,3])

        # plot plotly gauge chart on predicted value ---------------------------------------------------------------------------------------


        with col1:
            head1 ='<p style="margin-top: 25px; font-weight: bold; font-family:courier; font-size: 18px; text-align: center;">Predicted Credit Default Probability</p>'
            st.markdown(head1, unsafe_allow_html=True)
            layout = go.Layout(autosize=False, width=360, height=150,
                              margin= go.layout.Margin(t=0, b=25))
            fig = go.Figure(layout=layout)
            fig.add_trace(go.Indicator(value = proba,
                                       number = {'valueformat': '.2%', 'font':{'size':24}},
                                       gauge = {'axis': {'range': [0, 1],'dtick': 0.76},
                                                'bar': {'color': 'blue'},
                                                'steps': [
                                                    {'range': [0, 0.76], 'color': 'red'},
                                                        {'range': [0.76, 1], 'color': 'green'}
                                                ]}
                                                    ))

            fig.update_layout(
                              grid = {'rows': 1, 'columns': 1},
                              template = {'data' : {'indicator': [{
                                                                   'mode' : "number+gauge"}]}})
                                                                   #'domain': {'x': [0, 1], 'y': [0, 1]}


            col1.plotly_chart(fig, use_contrainer_width=True)

        #graphic 2
        with col2:
            head2 ='<p style="margin-top: 25px; font-weight: bold; font-family:courier; font-size: 18px; text-align: center;">Revenue Amounts by Revenue Types</p>'
            st.markdown(head2, unsafe_allow_html=True)

            #def revenu_plot():
            fig, ax= plt.subplots(constrained_layout=True)
            infog= client_info.loc[:, ['AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE']].groupby('NAME_INCOME_TYPE').mean()
            infog.plot(kind= 'bar', ax=ax, label='Type', color='b')

            ax.set_xlabel('Client Income Types')
            ax.set_ylabel('Total Revenus')
            ax.set_xticks(range(0,len(infog.index)), list(infog.index), rotation=30)
            ax.set_ylim(0, 600000)
            ax.legend(['Type'])

            #add client
            tc = Revenue
            ax.axhline(tc, color='orange', label='Client')

            #get the two labels as legends
            handles,_ = ax.get_legend_handles_labels()
            plt.legend(handles= handles, labels = ['Client', 'Type'])
            fig.set_tight_layout(True) # to prevent axis labels overlapping
            #ax.figure.savefig(plt.gcf())
            #plt.savefig()

            #return  plt.gcf()#plotly.io.to_image(plt.gcf(), engine='kaleido', format='jpg')
            #plotly.io.to_image(fig, engine='kaleido', validate=False)
            #fig= revenu_plot()
            #st.pyplot(plt.gcf(), use_column_width=True) #return oversize figure

            #st.pyplot(fig)
            st_shap(plt.gcf(), height=280, width=1000)
            #st.image(fig)
            #col2.plotly_chart(fig, use_contrainer_width=True)

        # plot shap value -------------------------------------------------------------------------------------------------------
        # Shap Waterfall_plot on class 0
        head3 ='<p style="margin-top: 25px; font-weight: bold; font-family:courier; font-size: 18px; text-align: center;">Default Probability Influence Factors</p>'
        st.markdown(head3, unsafe_allow_html=True)

        explainer, sv_explainer, pd_inputs = get_shap_data()

        shap.initjs()
        fig = plt.figure()
        #shap.waterfall_plot(shap.Explanation(sv_explainer[0,:,0], sv_explainer.base_values[:,0], pd_inputs, feature_names=pd_inputs.columns), show=False)
        st_shap(shap.waterfall_plot(shap.Explanation(sv_explainer[0,:,0], sv_explainer.base_values[:,0], pd_inputs, feature_names=pd_inputs.columns)), height=390, width=1100)
        #plt.gcf().set_size_inches(8,0.8)
        #plt.tight_layout()
        #st.pyplot(plt.gcf())

        # hide streamlit style
        hide_st_style = """
                    <style>
                    MainMenu {visibility: hidden;}
                    footer {visibility: hidden;}
                    header {visibility: hidden;}
                    </style>
                    """
        st.markdown(hide_st_style, unsafe_allow_html= True)

if __name__ == '__main__':
    main()


Writing dashboard.py
